In [2]:
import os

from json import loads
from os import getenv
#from keras import models
from firebase_admin import credentials, firestore, initialize_app

In [4]:
cred = credentials.Certificate(os.path.abspath("C:/Users/ASUS/Documents/GitHub/backend/src"))
db = firestore.client()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\ASUS\\Documents\\GitHub\\backend\\src'